# Jordan-Wigner transform

Load `QuantumOps`, the implementation of Pauli and Fermionc operators

In [1]:
using QuantumOps

Load some specific identifiers

In [2]:
using ElectronicStructure: MolecularData, Geometry, Atom, MolecularSpec, InteractionOperator
using ElectronicStructurePySCF: PySCF

## Specify geometry of molecules

Define geometries for three molecules: specification of atoms and their positions.
These are molecular hydrogen, `LiH`, and water.

In [3]:
geoms = (
    Geometry(Atom(:H, (0., 0., 0.)), Atom(:H, (0., 0., 0.7414))),

    Geometry(Atom(:Li, (0., 0., 0.)), Atom(:H, (0., 0., 1.4))),

    Geometry(Atom(:O, (0., 0., 0.)), Atom(:H, (0.757, 0.586, 0.)),
             Atom(:H, (-0.757, 0.586, 0.)))
);

Choose one of the geometries, 1, 2, or 3.

In [4]:
geom = geoms[1]

ElectronicStructure.Geometry{Float64}(ElectronicStructure.Atom{Float64}[ElectronicStructure.Atom{Float64}(:H, (0.0, 0.0, 0.0)), ElectronicStructure.Atom{Float64}(:H, (0.0, 0.0, 0.7414))])

Choose a orbital basis set.

In [5]:
basis = "sto-3g";
# basis = "631g"

We have chosen the crudest basis set and the smallest molecule. Otherwise
the size of the data structures is too large to display in a demonstration.

## Compute interaction integrals for the chosen molecule and basis set

Construct the specification of the electronic structure problem.

In [6]:
mol_spec = MolecularSpec(geometry=geom, basis=basis)

ElectronicStructure.MolecularSpec{Float64}(ElectronicStructure.Geometry{Float64}(ElectronicStructure.Atom{Float64}[ElectronicStructure.Atom{Float64}(:H, (0.0, 0.0, 0.0)), ElectronicStructure.Atom{Float64}(:H, (0.0, 0.0, 0.7414))]), 1, 0, "sto-3g")

Do calculations using `PySCF` and populate a `MolecularData` object with results.
`mol_pyscf` holds a constant, a rank-two tensor, and a rank-four tensor.

In [7]:
mol_pyscf = MolecularData(PySCF, mol_spec);

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall ~/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


## Include spin orbitals and change the representation

Create an interaction operator from one- and two-body integrals and a constant.
This does just a bit of manipulation of `mol_data`; converting space orbitals
into space-and-spin orbitals. There are options for choosing chemists' or physicists'
index ordering and block- or inteleaved-spin orbital ordering.
We take the default here,
which gives the same as the operator by the same name in OpenFermion.
The data is still in the form of rank two and four tensors, but the size of each
dimension is doubled.

In [8]:
iop = InteractionOperator(mol_pyscf);

Now we convert `iop` to a more-sparse format; a `FermiSum` (alias for `OpSum{FermiOp}`).
This is sparse in the sense that only non-zero entries in the tensors in `iop` are represented.
However, it is not as sparse as it could be, in that identity operators on modes are represented
explicitly.

In [9]:
fermi_op = FermiSum(iop)

25x4 QuantumOps.FermiSum{Vector{Vector{QuantumOps.FermiOps.FermiOp}}, Vector{Float64}}:
IIII * 0.7137539936876182
IIIN * -0.47594871522096427
IINI * -1.2524635735648983
IINN * 0.48217928821207207
II+- * 1.942890293094024e-16
INII * -0.47594871522096427
ININ * 0.6973937674230277
INNI * 0.663468096423568
IN+- * 1.249000902703301e-16
IN-+ * -1.249000902703301e-16
NIII * -1.2524635735648983
NIIN * 0.663468096423568
NINI * 0.6744887663568377
NI+- * -6.938893903907228e-18
NI-+ * 6.938893903907228e-18
NNII * 0.48217928821207207
+-II * 1.942890293094024e-16
+-IN * 1.249000902703301e-16
+-NI * -6.938893903907228e-18
+-+- * 0.18128880821149598
+--+ * -0.18128880821149598
-+IN * -1.249000902703301e-16
-+NI * 6.938893903907228e-18
-++- * -0.18128880821149598
-+-+ * 0.18128880821149598

## Jordan-Wigner transform

In [10]:
using QuantumOps: jordan_wigner

Compute the Jordan-Wigner transform

In [11]:
jordan_wigner(fermi_op)

15x4 QuantumOps.PauliSum{Vector{Vector{QuantumOps.Paulis.Pauli}}, Vector{ComplexF64}}:
IIII * (0.9373260195928107 + 0.0im)
IIIZ * (0.16016101508151956 + 0.0im)
IIZI * (-0.37553273780506796 + 0.0im)
IIZZ * (0.04283503098536895 + 0.0im)
IZII * (0.09925152906989478 + 0.0im)
IZIZ * (0.014034240457377972 + 0.0im)
IZZI * (0.01335152568887115 + 0.0im)
XXXX * (0.045322202052873996 + 0.0im)
XXYY * (0.045322202052873996 + 0.0im)
YYXX * (0.045322202052873996 + 0.0im)
YYYY * (0.045322202052873996 + 0.0im)
ZIII * (-0.23512989242032395 + 0.0im)
ZIIZ * (0.02094407108621566 + 0.0im)
ZIZI * (0.021291966781192325 + 0.0im)
ZZII * (0.015221225169759003 + 0.0im)

# Other features

An optional second argument of type `<:AbstractPauli` determines the output type.
The default `AbstractPauli` for all of `QuantumOps` is

In [12]:
QuantumOps.PauliDefault

QuantumOps.Paulis.Pauli

Here, we choose `PauliI` instead

In [13]:
jordan_wigner(fermi_op, PauliI)

15x4 QuantumOps.OpSum{QuantumOps.PaulisI.PauliI, Vector{Vector{QuantumOps.PaulisI.PauliI}}, Vector{ComplexF64}}:
IIII * (0.7639514007071802 + 0.0im)
IIIZ * (0.07473754924256235 + 0.0im)
IIZI * (-0.1156953846776437 + 0.0im)
IIZZ * (0.005408790196688342 + 0.0im)
IZII * (0.04502875706477448 + 0.0im)
IZIZ * (0.0017721070922012566 + 0.0im)
IZZI * (0.001685900525704425 + 0.0im)
XXXX * (0.045322202052873996 + 0.0im)
XXYY * (0.045322202052873996 + 0.0im)
YYXX * (0.045322202052873996 + 0.0im)
YYYY * (0.045322202052873996 + 0.0im)
ZIII * (-0.07039026960008568 + 0.0im)
ZIIZ * (0.00264461315339215 + 0.0im)
ZIZI * (0.002688542030789311 + 0.0im)
ZZII * (0.0019219879520549212 + 0.0im)

We have mentioned before that the performance of `PauliI` vs. `Pauli` may vary greatly
in general. But, in most cases, overall performance does not hinge on choosing beteen the two.

We won't use it now, but let's look at a sparser representation, backed by a `SparseVector`.
Here, only non-identity operators are represented explicitly.

In [14]:
using QuantumOps: sparse_op
sparse_op(fermi_op)

25x4 QuantumOps.FermiSum{Vector{SparseArraysN.SparseVector{QuantumOps.FermiOps.FermiOp, Int64}}, Vector{Float64}}:
0.7137539936876182
N4 * -0.47594871522096427
N3 * -1.2524635735648983
N3 N4 * 0.48217928821207207
+3 -4 * 1.942890293094024e-16
N2 * -0.47594871522096427
N2 N4 * 0.6973937674230277
N2 N3 * 0.663468096423568
N2 +3 -4 * 1.249000902703301e-16
N2 -3 +4 * -1.249000902703301e-16
N1 * -1.2524635735648983
N1 N4 * 0.663468096423568
N1 N3 * 0.6744887663568377
N1 +3 -4 * -6.938893903907228e-18
N1 -3 +4 * 6.938893903907228e-18
N1 N2 * 0.48217928821207207
+1 -2 * 1.942890293094024e-16
+1 -2 N4 * 1.249000902703301e-16
+1 -2 N3 * -6.938893903907228e-18
+1 -2 +3 -4 * 0.18128880821149598
+1 -2 -3 +4 * -0.18128880821149598
-1 +2 N4 * -1.249000902703301e-16
-1 +2 N3 * 6.938893903907228e-18
-1 +2 +3 -4 * -0.18128880821149598
-1 +2 -3 +4 * 0.18128880821149598

You can also perform the transform on a single term (`OpTerm`). This always returns
a sum (`OpSum{<:AbstractPauli}`).
Here is a single term.

In [15]:
fermi_op[2]

4-factor QuantumOps.FermiTerm{Vector{QuantumOps.FermiOps.FermiOp}, Float64}:
IIIN * -0.47594871522096427

The transform is

In [16]:
jordan_wigner(fermi_op[2])

2x4 QuantumOps.PauliSum{Vector{Vector{QuantumOps.Paulis.Pauli}}, Vector{ComplexF64}}:
IIII * (-0.03004908232578217 - 0.0im)
IIIZ * (0.03004908232578217 - 0.0im)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*